# Forest Fire Detection with transfer learning using VGG19

In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.models import load_model
from keras.applications import VGG19
from keras import layers
from keras import Model
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

train_dir = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Original/Training"
test_dir = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Original/Test"

#train_dir = "data/Training"
#test_dir = "data/Test"

VGG_SIZE = 224

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE),
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='binary')

FileNotFoundError: ignored

In [ ]:
transfer_model = VGG19(include_top=True,
                        weights="imagenet",
                        classes=1000)
input = transfer_model.inputs

In [ ]:
x = transfer_model.get_layer("block5_pool").output
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dropout(0.15)(x)
x = layers.Dense(1024, activation="relu")(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

classifier = Model(input, output_layer)

for layer in classifier.layers[:-4]:
    layer.trainable = False

classifier.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
classifier.compile(loss="binary_crossentropy",
                optimizer=optimizers.Adam(lr=0.001),
                metrics=["accuracy"])

In [ ]:
callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath='models/vgg19_model.h5', save_best_only=True, monitor='val_accuracy'),
]

history = classifier.fit(train_generator, 
                        epochs=5, 
                        batch_size=40,
                        validation_data=test_generator,
                        callbacks=callbacks)

Epoch 1/5
1478/1478 [==============================] - 5567s 4s/step - loss: 0.0492 - accuracy: 0.9835 - val_loss: 0.7559 - val_accuracy: 0.7159
Epoch 2/5
1478/1478 [==============================] - 5664s 4s/step - loss: 0.0476 - accuracy: 0.9833 - val_loss: 1.1254 - val_accuracy: 0.7694
Epoch 3/5
1478/1478 [==============================] - 5824s 4s/step - loss: 0.0463 - accuracy: 0.9843 - val_loss: 1.0169 - val_accuracy: 0.7390
Epoch 4/5
1478/1478 [==============================] - 5683s 4s/step - loss: 0.0390 - accuracy: 0.9866 - val_loss: 1.3191 - val_accuracy: 0.6860


In [ ]:
#classifier.save("models/vgg19_model.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
result = classifier.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

431/431 [==============================] - 1247s 3s/step - loss: 1.2371 - accuracy: 0.7248
Test loss: 1.2370550632476807, Test accuracy: 0.7248462438583374


In [ ]:
from keras import models

model_path = "/content/gdrive/My Drive/Escuela/Octavo Semestre/DSDM/models/vgg19_model_best.h5"
#model_path = 'models/vgg19_model.h5'

model = models.load_model(model_path)
result = model.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

431/431 [==============================] - 1280s 3s/step - loss: 0.7559 - accuracy: 0.7159
Test loss: 0.7558721899986267, Test accuracy: 0.7159104347229004
